*For demonstration, data from batch 1 is used*

In [ ]:
library(survival)

1. Import gene expression and clinical data

In [ ]:
# import read count data 
ori_data<-read.csv("/localhome/bs22tmhn/[ResearchProject]/Batch1/survData_batch1.csv",row.names = 1)

# import clinical data
clin<-read.csv("/localhome/bs22tmhn/[ResearchProject]/ClinData_batch1.csv",row.names = 1)

In [ ]:
# modify the ensembl ID names
colnames(ori_data)<-sub("\\..*", "",colnames(ori_data))

2. Perform PH assumption test (Schoenfeld method)

In [ ]:
#define a custom function to run PH assumption test
custom_cox.zph<-function(gene_expression, event_time, event_status){
  coxPH<-coxph(Surv(event_time, event_status) ~ gene_expression)
  zph <- cox.zph(coxPH)
  return(zph)
}

- OS

In [ ]:
#import candidate prognostic genes (50 genes in total)
candidates<-read.csv("/localhome/bs22tmhn/[ResearchProject]/candidate_genes_OS.csv")
genes<-array(candidates$Variable)

#extract 10 random gene names from the candidate genes
set.seed(123)
genes_10<-sample(as.vector(genes),size=10)

# select those 10 genes from the ori_data
reads<-ori_data[!is.na(ori_data$status),colnames(ori_data) %in% genes_10]

#combine with clinical data
clinOS<-clin[!is.na(clin$status),]
surv<-cbind(reads,data.frame(OS_time=clinOS[,c('OS_time')],status=clinOS[,c('status')]))

#run PH assumption test
zph<-apply(surv[,1:10], MARGIN=2, custom_cox.zph, event_time = surv$OS_time, event_status = surv$status)

In [ ]:
# draw a schoenfeld plot (e.g. for a gene with EnsemblID ENSG00000197912)
plot(zph$ENSG00000197912,ylab="Beta(t)")
title(main="ENSG00000197912",adj=0)

- TTT

In [ ]:
#import candidate genes 
candidates<-read.csv("/localhome/bs22tmhn/[ResearchProject]/candidate_genes_TTT.csv")
genes<-array(candidates$Variable)

#extract 10 random gene names from the candidate genes
set.seed(123)
genes_10<-sample(as.vector(genes),size=10)

# select those 10 genes from the ori_data
reads<-ori_data[,colnames(ori_data) %in% genes_10]

#combine with clinical data
surv<-cbind(reads,data.frame(trans_time=clin[,c('trans_time')],trans=clin[,c('trans')]))

#run PH assumption test
zph<-apply(surv[,1:10], MARGIN=2, custom_cox.zph, event_time = surv$trans_time, event_status = surv$trans)

In [ ]:
# draw a schoenfeld plot (e.g. for a gene with EnsemblID ENSG00000235703)
plot(zph$ENSG00000235703,ylab="Beta(t)")
title(main="ENSG00000235703",adj=0)

- PFS

In [ ]:
#import candidate genes 
candidates<-read.csv("/localhome/bs22tmhn/[ResearchProject]/candidate_genes_PFS.csv")
genes<-array(candidates$Variable)

#extract 10 random gene names from the candidate genes
set.seed(123)
genes_10<-sample(as.vector(genes),size=10)

# select those 10 genes from the ori_data
reads<-ori_data[!is.na(ori_data$status),colnames(ori_data) %in% genes_10]

#combine with clinical data
surv<-cbind(reads,data.frame(prog_time=clinOS[,c('prog_time')],progressed=clinOS[,c('progressed')]))

#run PH assumption test
zph<-apply(surv[,1:10], MARGIN=2, custom_cox.zph, event_time = surv$prog_time, event_status = surv$progressed)

In [ ]:
# draw a schoenfeld plot (e.g. for a gene with EnsemblID ENSG00000141447)
plot(zph$ENSG00000141447,ylab="Beta(t)")
title(main="ENSG00000141447",adj=0)